In [1]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [3]:
encoder = LabelEncoder()

In [ ]:
data = pd.read_csv('city_market_tracker.tsv000', sep='\t')

In [ ]:
data.columns

In [ ]:
# PRICE_DROPS_YOY
data.isna().sum().max()

In [ ]:
nan_summary = data.isna().sum().sort_values(ascending=False)
nan_percent = (data.isna().mean() * 100).sort_values(ascending=False)
pd.concat([nan_summary, nan_percent], axis=1, keys=['NaN Count', '% Missing'])

In [ ]:
# Columns >70% empty - Drop (no columns)
# 40-70% empty - model based imputation
# Columns 0% - 40% empty - fill with mean

In [ ]:
imp = IterativeImputer(max_iter = 10, random_state = 0)

for col in data.columns:
    nan_percent = data[col].isna().mean() * 100
    
    if 0 < nan_percent <= 40:
        mean_value = data[col].mean()
        data[col].fillna(mean_value, inplace = True)

    elif 41 <= nan_percent <= 70:
        reshaped = data[[col]]
        data[col] = imp.fit_transform(reshaped)

In [ ]:
numeric_df = data.select_dtypes(include = ['number'])
numeric_df = numeric_df.dropna(axis = 1, how='all')
numeric_df = numeric_df.loc[:, numeric_df.nunique() > 1]

corr = numeric_df.corr()

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(48, 32))
ax = sns.heatmap(corr, 
            mask = mask, 
            cmap = 'coolwarm', 
            vmin = -1,
            vmax = 1,
            annot = True,
            annot_kws={"size": 14},
            fmt = ".2f", 
            linewidths = 0.5)
ax.set_title("Correlation Heatmap", fontsize=20, pad=20)
ax.tick_params(axis='both', labelsize=16)
ax.figure.savefig("correlation_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
corr_matrix = numeric_df.corr().abs()

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

reduced_df = numeric_df.drop(columns=to_drop)

In [ ]:
numeric_df1 = reduced_df.select_dtypes(include=['number'])

corr1 = numeric_df1.corr()

mask = np.triu(np.ones_like(corr1, dtype=bool))
plt.figure(figsize=(48, 32))
ax = sns.heatmap(corr1, 
            mask = mask, 
            cmap = 'coolwarm', 
            vmin = -1,
            vmax = 1,
            annot = True, 
            annot_kws={"size": 15},
            fmt = ".2f", 
            linewidths = 0.5)
ax.set_title("Correlation Heatmap (UPDATED)", fontsize=20, pad=20)
ax.tick_params(axis='both', labelsize=16)
ax.figure.savefig("correlation_heatmap_UPDATED.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
reduced_df.columns

In [ ]:
# Remove Outliers
for col in reduced_df.columns:
    Q1, Q3 = reduced_df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    reduced_df[col] = reduced_df[col].clip(lower, upper)

In [ ]:
# Recoimbine data
non_numeric_df = data.select_dtypes(exclude=['number'])
final_df = pd.concat([non_numeric_df, reduced_df], axis=1)

In [ ]:
len(final_df)

In [ ]:
final_df.columns

In [ ]:
final_df.head()

In [ ]:
data.to_csv("city_market_tracker_EDA.tsv", sep = "\t", index = False)

## XGBoost

In [ ]:
xgbData = pd.read_csv('city_market_tracker_EDA.tsv', sep='\t')

In [ ]:
xgbData = xgbData[xgbData['STATE'] == 'California']

In [ ]:
def encode_all_categoricals(df):
    
    df_encoded = df.copy()
    encoders = {}
    
    # Detect all non-numeric columns
    cat_cols = df_encoded.select_dtypes(include=['object', 'category']).columns
    
    for col in cat_cols:
        encoder = LabelEncoder()
        df_encoded[col] = pd.to_numeric(
            encoder.fit_transform(df_encoded[col].fillna('missing').astype(str)),
            downcast='integer'
        )
        encoders[col] = encoder
        
    return df_encoded, encoders

In [ ]:
xgbData.dtypes

In [ ]:
date_cols = ['PERIOD_BEGIN', 'PERIOD_END']

In [ ]:
xgbData[date_cols] = xgbData[date_cols].apply(pd.to_datetime, errors='coerce')

In [ ]:
xgbData1 = xgbData.copy()

In [ ]:
xgbData1 = xgbData1.loc[:, xgbData1.nunique(dropna=False) > 1]

In [ ]:
xgbData1.dtypes

In [ ]:
xgbData1['PERIOD_LENGTH'] = (xgbData1['PERIOD_END'] - xgbData1['PERIOD_BEGIN']).dt.days

In [ ]:
xgbData1['PERIOD_LENGTH'].unique()

In [ ]:
xgbData1 = xgbData1.drop(columns=['PERIOD_BEGIN', 'PERIOD_END'])

In [ ]:
cat_cols = xgbData1.select_dtypes(include='object').columns

In [ ]:
xgbData1_encoded, encoders = encode_all_categoricals(xgbData1)

In [ ]:
xgbData1_encoded.select_dtypes(include='object').columns

In [ ]:
target = 'MEDIAN_LIST_PRICE'
X = xgbData1_encoded.drop(columns=[target])
y = xgbData1_encoded[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

In [ ]:
plt.figure(figsize=(10, 8))
plot_importance(model, max_num_features=15)
plt.show()

In [ ]:
param_dist = {
    'n_estimators': randint(200, 800),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10)
}

random_search = RandomizedSearchCV(
    XGBRegressor(tree_method='hist', random_state=42),
    param_distributions=param_dist,
    scoring='r2',
    n_iter=30,
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print("Best Parameters:", random_search.best_params_)
print("Best R²:", random_search.best_score_)

In [ ]:
best_model = random_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {rmse:.2f}")
print(f"Test R²: {r2:.3f}")

In [ ]:
pip install --upgrade scikit-learn

## Prepare for D3

In [ ]:
X_full = xgbData1_encoded.drop(target, axis=1)
y_full = xgbData[target]

In [ ]:
model = xgb.XGBRegressor(
    n_estimators=227,
    learning_rate=0.1658833703545386,
    max_depth=7,
    subsample=0.6206199253172188,
    colsample_bytree=0.7062755635564497,
    random_state=42
)
model.fit(X_full, y_full)

In [ ]:
y_full_pred = model.predict(X_full)

In [ ]:
full_df = xgbData.copy()
full_df["predicted_price"] = y_full_pred

In [ ]:
full_df.to_csv("xgb_predicted_prices.tsv", index=False) #Christian Provided RF

In [ ]:
full_df.info()

In [ ]:
state_avg = full_df.groupby(["STATE", "STATE_CODE"])["predicted_price"].mean().reset_index()
state_avg.to_csv("xgb_state_predictions.tsv", index=False) #rf_state_predictions_with_preds.tsv

In [ ]:
cities = pd.read_csv("uscities.csv")

In [ ]:
df_cities = full_df.merge(cities[["city", "state_id", "lat", "lng"]], left_on=["CITY", "STATE_CODE"], right_on=["city", "state_id"])
df_cities.to_csv("city_predictions.tsv", index=False)